In [1]:
from netgen.csg import *
from ngsolve import *
from ngsolve.utils import (
    Grad
)  # If I don't import these explicitly, VSCode reads them as missing.
import netgen.gui  # this opens up the netgen ui
import Magnetisation_Functions as magfunc
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
import Elastic_Functions as elfunc
import General_Functions as genfunc
import QMatrix
import sys
import time
np.set_printoptions(threshold=sys.maxsize)


# MATERIAL PARAMETERS
M_s = 1.5e6
gyromagnetic_ratio = 1.761e11
exchange_length = 3e-9
lambda_m: float = 30e-6  # saturation magnetostrain parameter
mu_0 = 1.25663706e-6
density = 7874
KAPPA: float = genfunc.calculate_KAPPA(density, exchange_length, gyromagnetic_ratio, mu_0)  # Determines the relative strength of the elastic vs. magnetic parts.
stress_fac = genfunc.stress_density_factor(KAPPA=KAPPA, mu_0=mu_0, M_s=M_s)
mu, lam = 54e9/stress_fac, 172e9/stress_fac
ALPHA: float = 1  # Dissipative constant in the LLG equation.

# SIMULATION PARAMETERS
T_MAX_DIM: float = 1e-10  # The maximum time for the simulation in seconds
THETA: float = 0.505  # Should be strictly above 1/2 for unconditional stability
time_step_dim = 1e-12 # seconds
K: float = time_step_dim*genfunc.nondimensional_time(gyromagnetic_ratio, mu_0, M_s=M_s)  # TIME STEP
T_MAX: float = T_MAX_DIM*genfunc.nondimensional_time(gyromagnetic_ratio, mu_0, M_s=M_s)
H_MAX: float = 6  # Determines how fine the mesh should be.

print(f"µ = {mu}, λ = {lam}, κ = {KAPPA}")
print(f"K = {K}, T_MAX = {T_MAX}")

µ = 6.915679893182856, λ = 22.027721141249096, κ = 2761.6363810703906
K = 0.331940679399, T_MAX = 33.1940679399


In [2]:
def MakeGeometry():  # this makes a box, with labelled faces
    geometry = CSGeometry()
    left = Plane(Pnt(0, 0, 0), Vec(-1, 0, 0)).bc("left")
    right = Plane(Pnt(6, 6, 6), Vec(1, 0, 0)).bc("right")
    front = Plane(Pnt(0, 0, 0), Vec(0, -1, 0)).bc("front")
    back = Plane(Pnt(6, 6, 6), Vec(0, 1, 0)).bc("back")
    bot = Plane(Pnt(0, 0, 0), Vec(0, 0, -1)).bc("bot")
    top = Plane(Pnt(6, 6, 6), Vec(0, 0, 1)).bc("top")

    cube = left * right * front * back * bot * top
    geometry.Add(cube)
    # cube = OrthoBrick(Pnt(0,0,0), Pnt(1,1,1))
    return geometry


ngmesh = MakeGeometry().GenerateMesh(maxh=H_MAX)
# ngmesh.Save("cube.vol")
mesh = Mesh(ngmesh)
Draw(mesh)

In [3]:
fes_mag = VectorH1(
    mesh, order=1
)  # the finite element space for the magnetisation m_h^i
fes_eps_m = MatrixValued(
    H1(mesh, order=1), dim=3
)  # matrix FE space on the magnetic part
fes_disp = VectorH1(
    mesh, order=1, dirichlet="left"
)  # the finite element space for the displacement u_h^i

print(f"mag_ndof={fes_mag.ndof}, disp_ndof={fes_disp.ndof},\n, dispfree_ndof={fes_disp.FreeDofs()}")
mag_gfu = GridFunction(fes_mag)
disp_gfu = GridFunction(fes_disp)
prev_disp_gfu = GridFunction(fes_disp) #  used to store the previous displacement
zeeman_factor = Parameter(0)
f_zeeman = CoefficientFunction((0, 0, zeeman_factor))
# body force and traction force
body_factor = Parameter(genfunc.force_density_grav(grav_accel=-9.81, density=7874, exchange_length=3e-9, KAPPA=KAPPA, mu_0=1.25663706e-6, M_s=217.6))
body_factor = 0.0
f_body = CoefficientFunction((0.0, 0.0, body_factor))
surface_factor = Parameter(0)
g_surface = CoefficientFunction([(0,surface_factor,0) if bc=="top" else (0,0,0) for bc in mesh.GetBoundaries()])

mag_ndof=27, disp_ndof=27,
, dispfree_ndof=0: 001010111001010111001010111


In [4]:
# Initial conditions
mag_gfu = magfunc.give_random_magnetisation(mag_gfu)
#disp_gfu = elfunc.give_random_displacement(disp_gfu)
disp_gfu = elfunc.give_uniform_displacement(disp_gfu, (0.0, 0.0, 0.0))  # initial displacement
velocity_gfu = GridFunction(fes_disp)  # velocity
velocity_gfu = elfunc.give_uniform_displacement(velocity_gfu, (0.0, 0.0, 0.0))  # An initial velocity. Should only be used once in iteration.
Draw(mag_gfu, mesh, "magnetisation")
Draw(disp_gfu, mesh, "displacement")
proj_mag = magfunc.nodal_projection(mag_gfu, fes_mag)

a_mag_fixed = magfunc.build_fixed_mag(fes_mag, ALPHA, THETA, K)  # static portion of the magnetisation linear system

In [5]:
a_mag, f_mag = magfunc.build_magnetic_lin_system(
    fes_mag, mag_gfu, ALPHA, THETA, K, KAPPA, disp_gfu, mu, lam, lambda_m, f_zeeman
)
B = magfunc.build_tangent_plane_matrix(mag_gfu)
v = magfunc.give_magnetisation_update(a_mag, B, f_mag, a_mag_fixed)

gmres completed in 0.010967254638671875, info=0, residual = 3.0834709785487746e-16
stiffness max = 25.623121034315833
force max = 26.23057319839993


C:\Users\gpb22197\AppData\Roaming\Python\Python37\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:407: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


In [8]:
Q = QMatrix.qmatrix(mag_gfu)
print(Q.todense())



9
18
18
9
18
18
9
18
18
[[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.50626259  0.          0.
   0.          0.          0.          0.          0.          0.
  -0.86237938  0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.92787791  0.          0.          0.          0.          0.
   0.          0.          0.         -0.3215676   0.          0.
   0.          0.          0.          0.          0.          0.
  -0.1887773   0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.         -0.96767721  0.
   0.          0.          0.          0.          0.          0.
   0.         -0.25219202  0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.9887718   0.          0.          0.          0.
 

In [21]:
dotcheck = np.dot(Q,Q.T)
np.allclose(dotcheck.todense(), np.identity(18), atol=1e-16)


True